In [1]:
from delfos.market.session import Session
import time

ModuleNotFoundError: No module named 'delfos'

In [ ]:
start_time = time.time()
p = Session(period=4)
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
print("-------- inativos --------")
print(p.inactive_tickers)
print(len(p.inactive_tickers))
print()
print("-------- ativos --------")
print(p.active_tickers)
print(len(p.active_tickers))


In [ ]:
start_time = time.time()
p.update_allocation_model()
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
import pandas as pd
from pypfopt import risk_models, expected_returns

df = pd.DataFrame({"weights": p.buy_weights})
stocks = p.select_stocks(df.index.tolist())
df["companies"] = [s.company for s in stocks]
df["mcap rets"] = round(p.market_implied_rets *100, 2)
df["market implied rets"] = df["mcap rets"].astype(str) + "%"
df["historical rets"] = expected_returns.mean_historical_return(p.closing_prices_table, compounding=True, frequency=248)
df["historical rets"] = round(df["historical rets"] * 100, 2)
df["mcap"] = [s.market_cap for s in stocks]
df["mcap"] = df["mcap"].map('{:,.0f}'.format)
df["sector"] = [s.sector for s in stocks]
df["sub sector"] = [s.sub_sector for s in stocks]
df["segment"] = [s.segment for s in stocks]

df["hist rets float"] = df["historical rets"]
df["historical rets"] = df["historical rets"].astype(str) + "%"

df = df.sort_values(by=['weights'], ascending=False)

In [ ]:
df = df.sort_values(by=['hist rets float'], ascending=False)
df[df["weights"] > 0]

In [ ]:
#rets = expected_returns.returns_from_prices(p.closing_prices_table, log_returns=True)
hist_rets = expected_returns.mean_historical_return(p.closing_prices_table, returns_data=False, compounding=True, frequency=248)

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
plt.style.use('fivethirtyeight')

ticker = "BAHI3"
clean = False

print(p.stocks[ticker].company)
print()

p.stocks[ticker].history["Rets"] = p.stocks[ticker].history["Close"].pct_change().dropna(how="all")
rets = p.stocks[ticker].history["Rets"].dropna()

p.stocks[ticker].history["Close"].plot()
print("---- Real ----")    
print("Maior retorno:", round(p.stocks[ticker].history["Close"].pct_change().dropna(how="all").max() *100, 2), "%")
print("Menor retorno:", round(p.stocks[ticker].history["Close"].pct_change().dropna(how="all").min() *100, 2), "%")
print("Retorno médio:", round(p.stocks[ticker].history["Close"].pct_change().dropna(how="all").mean() *100, 2), "%")
print("Desvio padrão:", round(p.stocks[ticker].history["Close"].pct_change().dropna(how="all").std() *100, 2), "%")

if clean:
    rets = rets.dropna().sort_values(ascending=False)
    print((rets*100).tail(20))
print("Tamanho da amostra:", len(p.stocks[ticker].history["Close"]))
print()


if clean:
    low=0.01
    high=0.99
    rets = pd.DataFrame(rets)
    quant_df = rets.quantile([low, high])
    rets = rets.apply(lambda x: x[(x>quant_df.loc[low,x.name]) & 
                                  (x < quant_df.loc[high,x.name])], axis=0)
    
    rets = rets["Rets"]
    p.stocks[ticker].history["New Rets"] = rets
    
    close = p.stocks[ticker].history["Close"][~p.stocks[ticker].history["New Rets"].isna()]
    close.plot()
    print("---- Limpo ----")
    print("Maior retorno:", round(rets.max() * 100, 2), "%")
    print("Menor retorno:", round(rets.min() * 100, 2), "%")
    print("Retorno médio:", round(rets.mean() * 100, 2), "%")
    print("Desvio padrão:", round(rets.std() * 100, 2), "%")
    print((rets*100).tail())
    
    print("Tamanho da amostra:", len(close))
    print()
    
    print("Diferença entre tamanho das amostras:", len(p.stocks[ticker].history["Close"]) - len(close))


    
    
#--------- TODO ------------
#Classe Stock:

    #CRIAR 2 COLUNAS NOVAS NO HISTÓRICO DA AÇÃO "cleaned rets" e "cleaned prices"
    #cleaned prices = history["Close"] onde cleaned rets é diferente de NaN 
    #criar um método auxiliar para transformar cada coluna de history em um atribtuto (pd.Series)
    
    
#--------- TODO ------------
#Classe Session:

    #decidir qual coluna de preços usar para a matriz de covariâncias
    #talvez separar a tablea de preços de fechamento em "closing_prices" e "cleaned_closing_prices"
    #separar a tabela de covariâncias em "shrinked_cov_table", "cov_table" e "corr_table"



In [ ]:
from scipy import stats
import statsmodels.api as sm
import pylab 
import numpy as np
import math

"""
if clean:
    low=0.0227
    high=0.9773
    rets = pd.DataFrame(rets)
    quant_df = rets.quantile([low, high])
    rets = rets.apply(lambda x: x[(x>quant_df.loc[low,x.name]) & 
                                  (x < quant_df.loc[high,x.name])], axis=0)
    rets = rets["Close"]
print(len(rets))
"""

#plotando para saber se a distribuição é (mais ou menos) normal
fig = sm.qqplot(rets, fit=True, line="s")
pylab.show()


#plotando histograma da distribuição
#(rets*100).plot.hist(bins=70)

mu, std = stats.norm.fit(rets) 
  
# Plot the histogram.
plt.hist(rets, bins=25, density=True, alpha=0.6, color='b')
  
# Plot the PDF.
xmin, xmax = plt.xlim()
x = np.linspace(xmin, xmax, 100)
pr = stats.norm.pdf(x, mu, std)
  
plt.plot(x, pr, 'k', linewidth=2)
title = "Fit Values: {:.2f} and {:.2f}".format(mu, std)
plt.title(title)
  
plt.show()


gmean = ((1 + rets).prod() ** (1 / rets.count()) - 1)
mean = rets.mean() * 1 
std = rets.std()  
skew = rets.skew() #> 0 = positive tail larger, < 0 = negative tail larger ... abs(skew) > 0.5 #significant skewness 
kurtosis = rets.kurtosis() #3 = Normal, >3 Fat tails (riskier)
#gstd = stats.gstd(rets+100)

print("------ Arithmetic ------")
print("mean: " + str(round(mean*100,2)) + "%")
print("std: " + str(round(std*100,2))+"%")
print("skewness: " + str(round(skew,2)))
print("kurtosis: " + str(round(kurtosis,2)))

print()
print("m + s: " + str(round(mean*100 + std*100, 2)) + "%")
print("m - s: " + str(round(mean*100 - std*100, 2)) + "%")
print()
print("------ Geometric ------")
print("gmean: " + str(round(gmean*100,2)) + "%")
#print("gstd: " + str(round(gstd*100,2)) + "%")


In [ ]:
hist = p.stocks[ticker].history
hist["Rets"] = hist["Rets"] *100
hist["Mov Avg"] = hist.rolling(window=14)["Rets"].mean()

In [ ]:
#hist["kurt"] = hist["Rets"].rolling(window=14).kurtosis()
import seaborn as sns
#hist["Mov Kurt"] = hist.rolling(window=14)["Rets"].skew()
#hist.plot.scatter(x="Volume", y="Rets")

hist["pe"] = hist["Close"] / p.stocks[ticker].bvps
sns.regplot(hist['pe'], hist['Rets'])


hist.tail(50)

In [ ]:
print(p.stocks["PETR4"].history["Close"].quantile(.05))
print(p.stocks["PETR4"].history["Close"].quantile(.95))

print(p.stocks["PETR4"].current_price)

In [ ]:
p.covariances_table

In [ ]:
corr_table = risk_models.cov_to_corr(p.covariances_table)
corr_table

In [ ]:
normal_corr_table = risk_models.cov_to_corr(p.closing_prices_table.cov())
normal_corr_table

In [ ]:
ticker1 = "VIVA3"
ticker2 = "CEAB3"
print(corr_table[ticker1][ticker2])
print(normal_corr_table[ticker1][ticker2])

In [ ]:
a = pd.Series([6, 19, 22, 15, 16])
b = pd.Series([6, -19, 22, 15, 16]) 

gsa = stats.gstd(a)
gsb = stats.gstd(b)
gma = stats.gmean(a)
gmb = stats.gmean(b)

print(gsa)
print(gsb)
print(gma)
print(gma)


In [ ]:
def gstd(x, m):
    return np.sqrt( (np.log(x / m)**2 ).sum() / x.count())

print(gstd(a, gma))
print(gstd(b, gmb))

In [121]:
import yfinance as yf
from datetime import datetime
import time


start_time = time.time()
now = datetime.now().date()

period = 6

ticker = "ITSA4"
ticker = ticker + ".SA"
stock = yf.Ticker(ticker)
#count=0

hist = stock.history(start=datetime(now.year-period,1,1), end=now)
#hist = stock.history(period="1d")

print("--- %s seconds ---" % (time.time() - start_time))
hist

hist = hist.drop(hist[hist.Open.isna() &
    hist.High.isna() &
    hist.Low.isna() & hist.Close.isna() &
    hist.Volume.isna()].index)
hist

--- 1.9781835079193115 seconds ---


,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2016-01-04,3.639564,3.650253,3.527331,3.527331,29060985.0,0.0,0.0
2016-01-05,3.575431,3.602154,3.538020,3.586121,15908450.0,0.0,0.0
2016-01-06,3.532676,3.687665,3.516642,3.618187,34541996.0,0.0,0.0
2016-01-07,3.548709,3.602153,3.521986,3.564742,23693412.0,0.0,0.0
2016-01-08,3.602153,3.628875,3.543364,3.564742,14933923.0,0.0,0.0
...,...,...,...,...,...,...,...
2022-01-10,9.023437,9.092469,8.954405,9.053022,21680100.0,0.0,0.0
2022-01-11,9.043161,9.220672,8.974130,9.200948,37639700.0,0.0,0.0
2022-01-12,9.210810,9.309426,9.072746,9.171363,31059000.0,0.0,0.0


In [112]:
len(hist[(hist["Close"].duplicated()) & (hist["Close"] == hist["Close"][-1])])

0

In [98]:
from bs4 import BeautifulSoup
import requests
import time
import pandas as pd

start_time = time.time()
#ticker = "BIDI11" + ".SA"
url = "https://finance.yahoo.com/quote/{}/history?p={}".format(ticker, ticker)

#opening up connection and grabbing page
response = requests.get(url,headers={'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.47 Safari/537.36'})
tables_list = pd.read_html(response.text)

soup = BeautifulSoup(response.content, "html.parser")
     
prices = soup.find_all("fin-streamer", {"data-field": "regularMarketPrice", "data-symbol":ticker})
for price in prices:
    print(price.text)

print("--- %s seconds ---" % (time.time() - start_time))
print(prices[0].text)
tables_list[0]
tables_list[0] = tables_list[0].drop(tables_list[0].head(1).index) #mantendo apenas a primeira row do histór

#tables_list[0] = tables_list[0].drop(tables_list[0].tail(len(tables_list[0])-1).index) #mantendo apenas a primeira row do histórico
tables_list[0]
df = tables_list[0]
#df = df.drop(df.tail(len(df)-2).index) #mantendo apenas a primeira row do histórico
#df = df.drop(df.head(1).index) #mantendo apenas a primeira row do histórico
df

19.73
--- 1.770784616470337 seconds ---
19.73


,Date,Open,High,Low,Close*,Adj Close**,Volume
1,"Jan 14, 2022",19.14,20.08,19.14,19.61,19.61,8234700
2,"Jan 13, 2022",19.64,19.88,19.19,19.30,19.30,5544700
3,"Jan 12, 2022",19.66,20.14,19.30,19.56,19.56,11896100
4,"Jan 11, 2022",19.00,19.56,18.85,19.45,19.45,10666400
5,"Jan 10, 2022",18.55,18.71,18.04,18.56,18.56,9375900
...,...,...,...,...,...,...,...
96,"Aug 30, 2021",29.27,29.32,28.58,28.74,28.43,7999900
97,"Aug 27, 2021",28.40,29.21,28.23,29.21,28.89,6838500
98,"Aug 26, 2021",28.63,29.20,27.88,28.03,27.72,13035000
99,"Aug 25, 2021",28.67,28.80,27.79,28.76,28.45,14538900


In [86]:
df = tables_list[0]
df = df.drop(columns=["Close*"])

df = df.rename(columns={"Adj Close**": "Close"})


df = df.drop(df[df.Open.str.contains("-") &
                df.Low.str.contains("-") &
                 df.High.str.contains("-") &
                 df.Close.str.contains("-") & df.Volume.str.contains("-") ].index)


df = df.drop(df.tail(1).index)
#df = df.drop(df.tail(len(df)-1).index)
df["Date"] = pd.to_datetime(df["Date"])
df = df.set_index("Date")

if df.index[1].date() == datetime.now().date():
        df = df.drop(df.tail(len(df)-2).index) #mantendo apenas a primeira row do histórico
        df = df.drop(df.head(1).index) #mantendo apenas a primeira row do histórico
else:
    df = df.drop(df.tail(len(df)-1).index) #mantendo apenas a primeira row do histórico

for col in df.columns:
    
    df[col] = df[col].str.replace("-", "0")
    df[col] = pd.to_numeric(df[col])
 


df

,Open,High,Low,Close,Volume
Date,,,,,
2022-01-14,19.14,20.08,19.14,19.61,8234700
